## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-12-23-49-14 +0000,nyt,Federal Judge Calls For Release of Some Immigr...,https://www.nytimes.com/2025/11/12/us/immigran...
1,2025-11-12-23-43-10 +0000,nyt,Former Syrian Security Official Indicted in Au...,https://www.nytimes.com/2025/11/12/world/europ...
2,2025-11-12-23-42-46 +0000,nyt,Virginia’s Governor-elect Wants a Say Over Lea...,https://www.nytimes.com/2025/11/12/us/virginia...
3,2025-11-12-23-39-43 +0000,nyt,Trump Summons Lauren Boebert as He Pushes GOP ...,https://www.nytimes.com/2025/11/12/us/politics...
4,2025-11-12-23-39-00 +0000,wsj,Biomining Startup Gets $13.5 Million as Critic...,https://www.wsj.com/articles/biomining-startup...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2326/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
18,trump,39
26,epstein,20
123,shutdown,18
162,new,16
66,house,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
228,2025-11-12-13-56-28 +0000,nypost,Furious White House reveals Jeffrey Epstein vi...,https://nypost.com/2025/11/12/us-news/jeffrey-...,133
126,2025-11-12-19-28-42 +0000,nypost,Michael Wolff told Epstein he should try to en...,https://nypost.com/2025/11/12/us-news/michael-...,107
217,2025-11-12-15-06-07 +0000,latimes,"Trump 'knew about the girls,' Jeffrey Epstein ...",https://www.latimes.com/politics/story/2025-11...,95
114,2025-11-12-20-02-00 +0000,wsj,Democrats Release New Epstein Emails Referring...,https://www.wsj.com/politics/policy/democrats-...,94
60,2025-11-12-21-50-30 +0000,nyt,New Epstein Emails Alleged Trump Knew of His C...,https://www.nytimes.com/2025/11/12/us/politics...,90


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
228,133,2025-11-12-13-56-28 +0000,nypost,Furious White House reveals Jeffrey Epstein vi...,https://nypost.com/2025/11/12/us-news/jeffrey-...
342,83,2025-11-12-02-00-00 +0000,wsj,As data went dark from the government shutdown...,https://www.wsj.com/economy/central-banking/th...
86,56,2025-11-12-21-02-48 +0000,nypost,Ex-top NY aide accused of being secret agent f...,https://nypost.com/2025/11/12/us-news/ex-top-n...
231,49,2025-11-12-13-51-00 +0000,nyt,India’s Prime Minister Vows Justice After New ...,https://www.nytimes.com/2025/11/11/world/asia/...
111,47,2025-11-12-20-09-00 +0000,wsj,Atlanta Fed President Raphael Bostic will reti...,https://www.wsj.com/economy/central-banking/at...
65,43,2025-11-12-21-40-32 +0000,nypost,NYC rabbis rip Michael Blake for stoking ‘anti...,https://nypost.com/2025/11/12/us-news/rabbis-c...
312,43,2025-11-12-07-18-55 +0000,nypost,Marriott guests left ‘homeless’ as rental comp...,https://nypost.com/2025/11/12/business/marriot...
172,37,2025-11-12-16-50-12 +0000,nypost,Why Jack Schlossberg can’t count on Kennedy fa...,https://nypost.com/2025/11/12/us-news/why-jack...
139,36,2025-11-12-18-48-19 +0000,wapo,Turkish military plane falls from sky over Geo...,https://www.washingtonpost.com/world/2025/11/1...
256,34,2025-11-12-11-03-00 +0000,wsj,Infineon Shares Climb After Chip Maker Forecas...,https://www.wsj.com/business/earnings/infineon...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
